
# <a href="https://github.com/khemanta"><img src="https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcSZFdXb_byVYeezcsUo2v9-vghc36hXO7MqzuAz6FLmvgmtqNDX" width=1200> </a>

# <center> GEOGRAPHIC SEGEMENTATION AND CLUSTERING OF NEIGHBORHOODS IN TORONTO, CANADA <center>
 > In this ecercise we will segmenting and clustering of the neighborhoods of Toronto City of Canada.


**EXECUTIVE SUMMARY**

**Motivation**: In this assignment, we are going to explore, segment, and cluster the geographic data of the neighborhoods in the city of Toronto. However, unlike New York, the neighborhood data is not readily available on the internet. What is interesting about the field of data science is that each project can be challenging in its unique way, so we need to learn to be agile and refine the skill to learn new libraries and tools quickly depending on the project.*


> **Data Source and Problem** : For the Toronto neighborhood data, a Wikipedia page [List of postal codes of Canada: M](https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M) exists that has all the information we need to explore and cluster the neighborhoods in Toronto. We will be required to scrape the Wikipedia page and wrangle the data, clean it, and then read it into a pandas dataframe so that it is in a structured format like the New York dataset in lab exercise.


**Strategy**: *Once the data is in a structured format, we can replicate the analysis that we did to the New York City dataset to explore and cluster the neighborhoods in the city of Toronto.* 

The reference for New York city based neighborhoods segmentation and clustering is here. 
[New York City Example](https://labs.cognitiveclass.ai/tools/jupyterlab/lab/tree/labs/DP0701EN/DP0701EN-3-3-2-Neighborhoods-New-York-py-v1.0.ipynb)


This work is partof submission will as a link to the Jupyter Notebook on my Github repository.

In [1]:
# Conditional installation of Python libraries and packages
# !pip install beautifulsoup4
# !pip install lxml

# !pip install requests
# !pip install html5lib

# !pip install geopy
# !pip install folium

# !pip install pandas
# !pip install numpy
# !pip install scipy

# !pip install scikit-learn

#!pip install -r requirement.txt

In [2]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation
from urllib.request import urlopen

#!pip install geopy
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 

# library for display html
from IPython.display import display_html
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize



import folium # plotting library for geographic color layers
from bs4 import BeautifulSoup
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

print('Requisite python lib packages installed ...')
print('and, all important libraries imported.')

Requisite python lib packages installed ...
and, all important libraries imported.


### 1. WEB-SCRAPING THE WIKIPEDIA PAGE FOR TABLE OF POSTAL CODE OF CANADA

* Using BeautifulSoup library of Python Wikipedia page of [List of postal codes of Canada: M](https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M), we will find the tabular data structure and read it and covert into a data frame. 

* The title of the webpage is printed to check if the page has been scraped successfully or not. Then the table of postal codes of Canada is printed.

In [3]:
src_url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
text_url = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text
soup = BeautifulSoup(text_url,'lxml')
#print(soup.prettify())

### 2. INSPECTING THE SOURCE-CODE OF THE WEBPAGE 
By Inspecting The Source Code of Table, the Class of Table is Wikitable Sortable. 
By inspecting the source code of the wikipage, the table is class wikitable sortable and the data of our interest as row have fields under <'tr'> ... <'/tr'> have all the records of the table which has three information fields. **PostalCode**, **Borough**, and **Neighborhood**

In [4]:
wiki_table = soup.body.table.tbody
#wiki_table

<img src="./images/toronto-webpage-1.png">

<img src="./images/toronto-webpage-2.png">

In [5]:
table = soup.find('table',{'class':'wikitable sortable'})
table_rows = table.find_all('tr')

In [6]:
print(soup.title)
from IPython.display import display_html
tab = str(soup.table)
#display_html(tab,raw=True)

dfs = pd.read_html(tab)
df=dfs[0]
#df.head()
df

<title>List of postal codes of Canada: M - Wikipedia</title>


,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
...,...,...,...
282,M8Z,Etobicoke,Mimico NW
283,M8Z,Etobicoke,The Queensway West
284,M8Z,Etobicoke,Royal York South West
285,M8Z,Etobicoke,South of Bloor


### 3. DATA PRE-PROCESSING, CLEANING AND CONEVRT TO DATAFRAME

As we see above lot of rows are having values 'Not assigned'. Lets remove those rows which have 'Not assigned' values to their corresponding columns.

In [7]:
#Data preprocessing and cleaning

# Dropping the rows where Borough is 'Not assigned'
df1 = df[df.Borough != 'Not assigned']

# Combining the neighbourhoods with same Postalcode
df2 = df1.groupby(['Postcode','Borough'], sort=False).agg(', '.join)
df2.reset_index(inplace=True)

# Replacing the name of the neighbourhoods which are 'Not assigned' with names of Borough
df2['Neighbourhood'] = np.where(df2['Neighbourhood'] == 'Not assigned',df2['Borough'], df2['Neighbourhood'])

df2

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Downtown Toronto,Queen's Park
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,Business Reply Mail Processing Centre 969 Eastern
101,M8Y,Etobicoke,"Humber Bay, King's Mill Park, Kingsway Park So..."


**Obserevation**: We are left with only 103 rows now after cleaning the data for 'Not assigned' as show below as well.

In [8]:
# information about the left over data 
df2.info()
# 
df2.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103 entries, 0 to 102
Data columns (total 3 columns):
Postcode         103 non-null object
Borough          103 non-null object
Neighbourhood    103 non-null object
dtypes: object(3)
memory usage: 2.5+ KB


(103, 3)

### In Order to create the above dataframe, we have accomplished under the followings guidelines :

* The dataframe will consist of three columns: **PostalCode**, **Borough**, and **Neighborhood**.
    
* Only process the cells that have an assigned borough. Ignore cells with a borough that is "Not assigned".

* More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.
* If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.

* Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.

* In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

### 4. READING THE LATITUDE AND LONGITUDE INFORMATION FROM GEOSPATIAL DATA
Importing the csv file conatining the latitudes and longitudes for various neighbourhoods in Canada

In [9]:
lat_lon = pd.read_csv('https://cocl.us/Geospatial_data')
lat_lon.head(11)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


### 5. MERGING OF THE NEIGHBORHOOD POSTAL CODE WITH GEO-CODE (LAT, LON)

#### Merging the Two tables for Getting the Latitudes and Longitudes for Various Neighbourhoods in Canada

In [10]:
lat_lon.rename(columns={'Postal Code':'Postcode'}, inplace=True)
df3 = pd.merge(df2,lat_lon,on='Postcode')
df3.head(11)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494
5,M9A,Queen's Park,Queen's Park,43.667856,-79.532242
6,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Woodbine Gardens, Parkview Hill",43.706397,-79.309937
9,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937


## 6. SEGMENTATION & CLUSTERING OF THE NEIGHBOURHOOD OF TORONTO 

The notebook from here includes the Clustering and the plotting of the neighbourhoods of Canada which contain Toronto in their Borough
Getting all the rows from the data frame which contains Toronto in their Borough.



### 6.1. SEGMENT THE DATA FOR BOROUGH WHICH ARE TORONTO IN CANADA

#### Getting all the rows from the data frame which contains Toronto in their Borough.


In [11]:
df4 = df3[df3['Borough'].str.contains('Toronto',regex=False)]
df4

,Postcode,Borough,Neighbourhood,Latitude,Longitude
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
4,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494
9,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
19,M4E,East Toronto,The Beaches,43.676357,-79.293031
20,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
24,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
25,M6G,Downtown Toronto,Christie,43.669542,-79.422564
30,M5H,Downtown Toronto,"Adelaide, King, Richmond",43.650571,-79.384568
31,M6H,West Toronto,"Dovercourt Village, Dufferin",43.669005,-79.442259


### 6.2. VISUALIZATION OF NEIGHBORHOODS DATA 

#### Visualizing all the Neighbourhoods of the above data frame using Folium


In [12]:
map_toronto = folium.Map(location=[43.651070,-79.347015], zoom_start=11)

for lat,lng,borough,neighbourhood in zip(df4['Latitude'],df4['Longitude'],df4['Borough'],df4['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
    [lat,lng],
    radius=5,
    popup=label,
    color='blue',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(map_toronto)
map_toronto

<a href="https://github.com/khemanta/Coursera_Capstone/blob/master/images/toronto-neighborhood-visualization.png"><img src="./images/toronto-neighborhood-visualization.png" width=1200> </a>

## 7. CLUSTERING OF THE NEIGHBORHOODS AND VISUALIZATION

### 7.1. USING K-MEANS CLUSTERING TO CLUSTER NEIGHBORHOODS

Using K-Means clustering for the clsutering of the neighbourhoods. For K-Means we need to give initial input of how many clusters we want, which is k=5 here.

In [13]:
k=5 # No. of culsters as input
toronto_clustering = df4.drop(['Postcode','Borough','Neighbourhood'],1)
kmeans = KMeans(n_clusters = k,random_state=0).fit(toronto_clustering)
kmeans.labels_
df4.insert(0, 'Cluster Labels', kmeans.labels_)

### 7.2. THE TORONTO BOROUGH DATA WITH COMPUTED CLUSTER LABELS

In [14]:
df4.head(11)

,Cluster Labels,Postcode,Borough,Neighbourhood,Latitude,Longitude
2,0,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
4,0,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494
9,0,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937
15,0,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
19,4,M4E,East Toronto,The Beaches,43.676357,-79.293031
20,0,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
24,0,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
25,3,M6G,Downtown Toronto,Christie,43.669542,-79.422564
30,0,M5H,Downtown Toronto,"Adelaide, King, Richmond",43.650571,-79.384568
31,1,M6H,West Toronto,"Dovercourt Village, Dufferin",43.669005,-79.442259


### 7.3. RE-VISIALIZING THE CLUSTERED NEIGHBORHOOD OF TORONTO BOROUGH

Visualization of the different clusters of the neighbourhood of borough which has Toronto with different color coded for different clusters.

In [15]:
# create map
map_clusters = folium.Map(location=[43.651070,-79.347015],zoom_start=10)

# set color scheme for the clusters
x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, neighbourhood, cluster in zip(df4['Latitude'], df4['Longitude'], df4['Neighbourhood'], df4['Cluster Labels']):
    label = folium.Popup('Cluster' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<a href="https://github.com/khemanta/Coursera_Capstone/blob/master/images/toronto-neighborhood-clustered-visualization.png"><img src="./images/toronto-neighborhood-clustered-visualization.png" width=1200> </a>

In [16]:
df2.shape

(103, 3)

In [17]:
df4.shape

(39, 6)